### Conjunto de datos de Finanzas

Este conjunto de datos contiene información sobre reservas de hoteles. A continuación, tenemos una descripción de lo que nos podemos encontrar en el conjunto de datos:

| Columna                       | Descripción                                                                                   |
|-------------------------------|-----------------------------------------------------------------------------------------------|
| `hotel`                       | Tipo de hotel                                                                                 |
| `is_canceled`                 | Indica si la reserva fue cancelada (`True`) o no (`False`)                                    |
| `lead_time`                   | Número de días entre la fecha de reserva y la fecha de llegada al hotel                       |
| `arrival_date_year`           | Año de llegada al hotel                                                                       |
| `arrival_date_month`          | Mes de llegada al hotel                                                                       |
| `arrival_date_week_number`    | Número de la semana de llegada al hotel                                                       |
| `arrival_date_day_of_month`   | Día del mes de llegada al hotel                                                               |
| `stays_in_weekend_nights`     | Número de noches que el cliente se quedó durante el fin de semana                             |
| `stays_in_week_nights`        | Número de noches que el cliente se quedó durante la semana                                    |
| `adults`                      | Número de adultos que acompañaban al cliente en la reserva                                    |
| `children`                    | Número de niños que acompañaban al cliente en la reserva                                      |
| `babies`                      | Número de bebés que acompañaban al cliente en la reserva                                      |
| `meal`                        | Tipo de comida incluida en la reserva (`BB`: Desayuno, `HB`: Media Pensión, `FB`: Pensión Completa) |
| `country`                     | País de origen del cliente                                                                    |
| `market_segment`              | Segmento de mercado al que pertenece la reserva                                               |
| `distribution_channel`        | Canal de distribución utilizado para realizar la reserva                                      |
| `is_repeated_guest`           | Indica si el cliente es un huésped repetido (`1`) o no (`0`)                                  |
| `previous_cancellations`      | Número de reservas canceladas por el cliente antes de esta reserva                            |
| `previous_bookings_not_canceled` | Número de reservas no canceladas por el cliente antes de esta reserva                    |
| `reserved_room_type`          | Tipo de habitación reservada                                                                  |
| `assigned_room_type`          | Tipo de habitación asignada en la reserva                                                     |
| `booking_changes`             | Número de cambios realizados en la reserva                                                    |
| `agent`                       | Identificador del agente involucrado en la reserva                                            |
| `company`                     | Identificador de la compañía involucrada en la reserva                                        |
| `days_in_waiting_list`        | Número de días que la reserva estuvo en lista de espera                                       |
| `customer_type`               | Tipo de cliente que realizó la reserva (`Transient`, `Contract`, `Group`, `Transient-Party`)  |
| `adr`                         | Tarifa promedio diaria pagada por la reserva                                                  |
| `required_car_parking_spaces` | Número de espacios de estacionamiento requeridos por el cliente                               |
| `total_of_special_requests`   | Número total de solicitudes especiales realizadas por el cliente                              |
| `reservation_status`          | Estado de la reserva (`Check-Out`: Salida, `Canceled`: Cancelada)                             |
| `reservation_status_date`     | Fecha del estado de la reserva                                                                |


In [11]:
# Libraries 
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np
import re
from word2number import w2n
from itertools import combinations
from scipy.stats import kstest, spearmanr, pearsonr, chi2_contingency

# Configuration
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
df = pd.read_csv('data/finanzas-hotel-bookings.csv', index_col=0)

In [13]:
df.head(2)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
0,Resort Hotel,False,342.0,2015.0,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,PRT,NaN,Direct,0.0,NaN,0.0,C,C,3.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN
1,Resort Hotel,False,737.0,NaN,July,27.0,1.0,0.0,0.0,2.0,NaN,0.0,BB,NaN,NaN,Direct,0.0,0.0,0.0,NaN,C,4.0,NaN,NaN,0.0,Transient,0.0,0.0,0.0,Check-Out,2015-07-01 00:00:00,NaN


In [14]:
df.columns

Index(['hotel', 'is_canceled', 'lead_time', 'arrival_date_year',
       'arrival_date_month', 'arrival_date_week_number',
       'arrival_date_day_of_month', 'stays_in_weekend_nights',
       'stays_in_week_nights', 'adults', 'children', 'babies', 'meal',
       'country', 'market_segment', 'distribution_channel',
       'is_repeated_guest', 'previous_cancellations',
       'previous_bookings_not_canceled', 'reserved_room_type',
       'assigned_room_type', 'booking_changes', 'agent', 'company',
       'days_in_waiting_list', 'customer_type', 'adr',
       'required_car_parking_spaces', 'total_of_special_requests',
       'reservation_status', 'reservation_status_date', '0'],
      dtype='object')

In [249]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 119562 entries, 0 to 182450
Data columns (total 31 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   hotel                           119390 non-null  object        
 1   is_canceled                     119390 non-null  object        
 2   lead_time                       119390 non-null  float64       
 3   arrival_date_year               64829 non-null   float64       
 4   arrival_date_month              119390 non-null  float64       
 5   arrival_date_week_number        101004 non-null  float64       
 6   arrival_date_day_of_month       119271 non-null  float64       
 7   stays_in_weekend_nights         119390 non-null  float64       
 8   stays_in_week_nights            119390 non-null  float64       
 9   adults                          119428 non-null  float64       
 10  children                        69302 non-null   float64     

In [5]:
# Diccionario de mapeo de meses a números
month_mapping = {
    'January': 1, 'February': 2, 'March': 3, 'April': 4, 'May': 5,
    'June': 6, 'July': 7, 'August': 8, 'September': 9, 'October': 10,
    'November': 11, 'December': 12, '1':1, '2':2, '3':3
}

# Aplicar el mapeo a la columna de meses
df.arrival_date_month = df.arrival_date_month.map(month_mapping).astype(float)


In [6]:
df.stays_in_weekend_nights = df.stays_in_weekend_nights.round()

In [7]:
df.stays_in_week_nights = df.stays_in_week_nights.round()

In [8]:
df.drop(columns=['0'], inplace=True)

In [9]:
# Eliminar filas donde todos los valores son NaN
df = df.dropna(how='all')

In [10]:
df.adr = abs(df.adr) # había 1 valor negativo 

In [104]:
# Filtramos el df dropando las 3 columnas que tenian valores mientras que todas las demas eran nan
df_filt = df.drop(columns=['lead_time', 'stays_in_weekend_nights', 'stays_in_week_nights']) 

# Guardamos en una variable los indices de las filas donde todo es nan, excepto en esas 3 columnas (son 100 filas, menos del 1% del total)
index = df_filt[df_filt.isnull().all(axis=1)].index 

# Dropamos estas 100 filas
df.drop(index=index, inplace=True)

In [217]:
df.reservation_status_date = pd.to_datetime(df.reservation_status_date, errors='coerce')

In [ ]:
# hotel --> NULOS
# is_canceled --> NULOS
# lead_time --> NULOS
# arrival_date_year --> NULOS
# arrival_date_month --> NULOS
# arrival_date_week_number --> NULOS
# arrival_date_day_of_month --> NULOS
# stays_in_weekend_nights --> NULOS
# stays_in_week_nights --> NULOS
# adults --> NULOS
# children --> NULOS
# babies --> NULOS
# meal --> Hay un valor 'Undefined' y NULOS.
# country --> NULOS
# market_segment --> Hay un valor 'Undefined' y NULOS.
# distribution_channel --> Hay un valor 'Undefined' y NULOS.
# is_repeated_guest --> NULOS
# previous_cancellations --> NULOS
# previous_bookings_not_canceled --> NULOS
# reserved_room_type --> NULOS (No tiene valores I, K)
# assigned_room_type --> NULOS (Tiene valores I, K)
# booking_changes --> NULOS
# agent --> NULOS (333 nunique)
# company --> NULOS (310 nunique)
# days_in_waiting_list --> NULOS
# customer_type --> NULOS
# adr --> NULOS, Hay 1 valor 5400 ojo outlier
# required_car_parking_spaces --> NULOS
# total_of_special_requests --> NULOS
# reservation_status --> NULOS
# reservation_status_date --> NULOS
# 0 --> OK (DROPPED)

In [ ]:
# Filtrar filas donde todos los valores son NaN
df[df.isnull().all(axis=1)]

In [32]:
df.tail(50)

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,market_segment,distribution_channel,is_repeated_guest,previous_cancellations,previous_bookings_not_canceled,reserved_room_type,assigned_room_type,booking_changes,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,0
182827,NaN,NaN,370.660078,NaN,NaN,NaN,NaN,15.787226,22.208331,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
182828,NaN,NaN,279.536555,NaN,NaN,NaN,NaN,17.024920,24.993146,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,TA/TO
182829,NaN,NaN,240.796119,NaN,NaN,NaN,NaN,11.403151,16.435737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
182830,NaN,NaN,267.532248,NaN,NaN,NaN,NaN,14.556978,16.326227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
182831,NaN,NaN,303.498507,NaN,NaN,NaN,NaN,14.006258,22.682888,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
182832,NaN,NaN,307.307332,NaN,NaN,NaN,NaN,11.054662,22.402225,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
182833,NaN,NaN,216.622471,NaN,NaN,NaN,NaN,13.164423,17.550158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,A
182834,NaN,NaN,316.159693,NaN,NaN,NaN,NaN,10.239304,20.764822,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
182835,NaN,NaN,245.661917,NaN,NaN,NaN,NaN,15.238687,16.416464,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0
182836,NaN,NaN,240.043225,NaN,NaN,NaN,NaN,18.420356,15.782039,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Conclusiones EDA

* hotel --> Categórica. Solo tiene 2 unique. NULOS
* is_canceled --> Binária. NULOS 
* lead_time --> Numérica. Verificar outliers. NULOS
* arrival_date_year --> Numérica. NULOS
* arrival_date_month --> Categórica. Hay que transformar los datos. NULOS.
* arrival_date_week_number --> Numérica. NULOS
* arrival_date_day_of_month --> Numérica. NULOS
* stays_in_weekend_nights --> Numérica. Usar round? NULOS
* stays_in_week_nights --> Numérica. Usar round? NULOS
* adults --> Numérica. NULOS
* children --> Numérica. NULOS
* babies --> Numérica. NULOS
* meal --> Categórica. Hay un valor 'Undefined' y NULOS.
* country --> Categórica. NULOS
* market_segment --> Categórica. Hay un valor 'Undefined' y NULOS.
* distribution_channel --> Categórica. Hay un valor 'Undefined' y NULOS.
* is_repeated_guest --> Binária. NULOS
* previous_cancellations --> Numérica. NULOS
* previous_bookings_not_canceled --> Numérica. NULOS
* reserved_room_type --> Categórica. NULOS (No tiene valores I, K)
* assigned_room_type --> Categórica. NULOS (Tiene valores I, K)
* booking_changes --> NUmérica. NULOS
* agent --> Numérica. NULOS (333 nunique)
* company --> Numérica. NULOS (310 nunique)
* days_in_waiting_list --> Numérica. NULOS
* customer_type --> Categórica. NULOS
* adr --> Numérica. Hay 1 valor negativo. NULOS. Hay 1 valor 5400 ojo outlier
* required_car_parking_spaces --> Numérica. NULOS
* total_of_special_requests --> Numérica. NULOS
* reservation_status --> Categórica. NULOS
* reservation_status_date --> Categórica. Cambiar tipo a datetime. NULOS
* 0 --> Dropar 

* arrival_date_year, arrival_date_month, arrival_date_day_of_month -- > Crear columna nueva con la fecha completa en formato datetime.